# ENTRENAMIENTO DEL MODELO
Vaig a fer rapid una base aon neteje rapid els nuls i prove diferents models para fer prediccions

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

In [39]:
data = pd.read_csv("temp.csv")
data = data[data["marca"].str.lower() != "thuraya"]
data

,_id,ano,memoria,marca,pantalla_in,pantalla_tipo,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
0,B00IRZ8EQC,2013.0,256.0,samsung,4.30,lcd,1.7,1.0,1.27,77.11,540.0,960.0,5000.0,279.459459,199.99,245.00
1,B00J8OA220,2024.0,160.0,inmarsat,2.00,lcd,2.9,2.0,10.16,318.00,240.0,320.0,160.0,0.800000,199.99,933.00
2,B00JC8MD7Y,2024.0,256.0,samsung,6.70,superamoled,2.2,6.0,0.77,179.00,1080.0,2400.0,4500.0,1761.818182,279.99,279.99
3,B00TKALUDC,2015.0,128.0,ttfone,2.00,lcd,0.0,32.0,1.80,75.00,1080.0,2400.0,800.0,168.571429,45.99,41.99
4,B00TUXHZTW,2015.0,128.0,doro,2.00,lcd,1.2,16.0,1.90,74.00,3840.0,2160.0,800.0,74.000000,48.67,48.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,B0DH3N1GZ3,2024.0,128.0,honor,6.56,lcd,22.1,4.0,2.57,254.42,1920.0,1080.0,5000.0,1.702128,199.99,99.90
937,B09QH71RZR,2024.0,128.0,xiaomi,6.43,amoled,2.0,6.0,0.81,179.00,1920.0,1080.0,5000.0,3821.818182,165.39,127.00
938,B0DK4NBZWB,2024.0,256.0,oppo,6.67,amoled,3.4,8.0,2.57,254.42,1080.0,2400.0,5000.0,2.291667,199.99,299.00
939,B0D8TCGKGN,2024.0,256.0,oukitel,6.52,hd,1.6,24.0,2.57,254.42,1280.0,720.0,10600.0,149.772727,199.99,149.99


In [45]:
def top_marcas(df):
    """Devuelve las 5 marcas más frecuentes en el dataset."""
    return df["marca"].value_counts().head(30)

top_marcas(data)

marca
92    194
70    107
18     79
61     60
58     41
50     39
9      32
68     31
84     29
4      24
13     19
33     19
87     16
28     15
60     13
85     12
78     10
51     10
57     10
94      9
25      8
74      8
62      6
82      6
30      5
93      5
35      5
10      5
90      4
48      4
Name: count, dtype: int64

## NETEJA DE NULS (RAPIDO)

In [18]:
def fill_na_with_mode(df):
    
    df_filled = df.copy()
    for column in df_filled.columns:
        mode_value = df_filled[column].mode()[0] 
        df_filled[column].fillna(mode_value, inplace=True)
    return df_filled

In [19]:
data = fill_na_with_mode(data)


C:\Users\Gerard\AppData\Local\Temp\ipykernel_21156\628468171.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled[column].fillna(mode_value, inplace=True)


## ELIMINAR IDS

In [20]:
def eliminar_columnas_no_deseadas(df):
    """Elimina columnas '_id' y cualquier columna no deseada como 'Unnamed'."""
    cols_a_eliminar = [col for col in df.columns if "Unnamed" in col or col == "_id"]
    df.drop(columns=cols_a_eliminar, inplace=True, errors='ignore')
    return df

In [21]:
data = eliminar_columnas_no_deseadas(data)
data


,ano,memoria,marca,pantalla_in,pantalla_tipo,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
0,2013.0,256.0,samsung,4.30,lcd,1.7,1.0,1.27,77.11,540.0,960.0,5000.0,279.459459,199.99,245.00
1,2024.0,160.0,inmarsat,2.00,lcd,2.9,2.0,10.16,318.00,240.0,320.0,160.0,0.800000,199.99,933.00
2,2024.0,256.0,samsung,6.70,superamoled,2.2,6.0,0.77,179.00,1080.0,2400.0,4500.0,1761.818182,279.99,279.99
3,2015.0,128.0,ttfone,2.00,lcd,0.0,32.0,1.80,75.00,1080.0,2400.0,800.0,168.571429,45.99,41.99
4,2015.0,128.0,doro,2.00,lcd,1.2,16.0,1.90,74.00,3840.0,2160.0,800.0,74.000000,48.67,48.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,2024.0,128.0,honor,6.56,lcd,22.1,4.0,2.57,254.42,1920.0,1080.0,5000.0,1.702128,199.99,99.90
937,2024.0,128.0,xiaomi,6.43,amoled,2.0,6.0,0.81,179.00,1920.0,1080.0,5000.0,3821.818182,165.39,127.00
938,2024.0,256.0,oppo,6.67,amoled,3.4,8.0,2.57,254.42,1080.0,2400.0,5000.0,2.291667,199.99,299.00
939,2024.0,256.0,oukitel,6.52,hd,1.6,24.0,2.57,254.42,1280.0,720.0,10600.0,149.772727,199.99,149.99


## CONVERTIR VARIABLES CATEGORICAS A NUMERICAS

In [41]:
def convertir_categoricas_a_numericas(df, categorical_cols):
    """Convierte variables categóricas en numéricas usando Label Encoding y guarda la referencia en un archivo de texto."""
    label_encoders = {}
    with open("referencia_marcas.txt", "w") as file:
        for col in categorical_cols:
            if col in df.columns:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
                label_encoders[col] = le
                for i, clase in enumerate(le.classes_):
                    file.write(f"{clase}:{i}\n")
    return df, label_encoders

In [44]:
# Definir columnas categóricas
categorical_cols = ["marca", "pantalla_tipo"]
df, label_encoders = convertir_categoricas_a_numericas(data, categorical_cols)
df

,_id,ano,memoria,marca,pantalla_in,pantalla_tipo,velocidad_cpu_ghz,ram,grosor,peso,ancho_px,alto_px,bateria,promedio_valoraciones,precio_anterior,precio_actual
0,B00IRZ8EQC,2013.0,256.0,70,4.30,7,1.7,1.0,1.27,77.11,540.0,960.0,5000.0,279.459459,199.99,245.00
1,B00J8OA220,2024.0,160.0,39,2.00,7,2.9,2.0,10.16,318.00,240.0,320.0,160.0,0.800000,199.99,933.00
2,B00JC8MD7Y,2024.0,256.0,70,6.70,10,2.2,6.0,0.77,179.00,1080.0,2400.0,4500.0,1761.818182,279.99,279.99
3,B00TKALUDC,2015.0,128.0,82,2.00,7,0.0,32.0,1.80,75.00,1080.0,2400.0,800.0,168.571429,45.99,41.99
4,B00TUXHZTW,2015.0,128.0,19,2.00,7,1.2,16.0,1.90,74.00,3840.0,2160.0,800.0,74.000000,48.67,48.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,B0DH3N1GZ3,2024.0,128.0,33,6.56,7,22.1,4.0,2.57,254.42,1920.0,1080.0,5000.0,1.702128,199.99,99.90
937,B09QH71RZR,2024.0,128.0,92,6.43,1,2.0,6.0,0.81,179.00,1920.0,1080.0,5000.0,3821.818182,165.39,127.00
938,B0DK4NBZWB,2024.0,256.0,58,6.67,1,3.4,8.0,2.57,254.42,1080.0,2400.0,5000.0,2.291667,199.99,299.00
939,B0D8TCGKGN,2024.0,256.0,61,6.52,4,1.6,24.0,2.57,254.42,1280.0,720.0,10600.0,149.772727,199.99,149.99


In [31]:
# Separar features y target
X = df.drop(columns=["precio_actual"])
y = df["precio_actual"]

In [32]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## GRADIENT BOSSTING REGRESSOR

In [33]:

# Entrenar modelo GradientBoostingRegressor
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)


# Entrenar modelo RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)


## EVALUACION

In [34]:

# Evaluar modelo GradientBoostingRegressor
gb_results = {
    "MAE": mean_absolute_error(y_test, y_pred_gb),
    "MSE": mean_squared_error(y_test, y_pred_gb),
    "R2 Score": r2_score(y_test, y_pred_gb),
}

# Evaluar modelo RandomForestRegressor
rf_results = {
    "MAE": mean_absolute_error(y_test, y_pred_rf),
    "MSE": mean_squared_error(y_test, y_pred_rf),
    "R2 Score": r2_score(y_test, y_pred_rf),
}

# Mostrar resultados
print("GradientBoostingRegressor:")
for metric, value in gb_results.items():
    print(f"  {metric}: {value}")

print("\nRandomForestRegressor:")
for metric, value in rf_results.items():
    print(f"  {metric}: {value}")

GradientBoostingRegressor:
  MAE: 56.06842695621778
  MSE: 9566.29774200081
  R2 Score: 0.7764515026956791

RandomForestRegressor:
  MAE: 45.94713518237083
  MSE: 8390.457699959123
  R2 Score: 0.8039289324765435


## GUARDAR MODELO

In [35]:
def guardar_modelo(model, filename="modelo_entrenado.pkl"):
    """Guarda el modelo entrenado en un archivo local."""
    with open(filename, "wb") as file:
        pickle.dump(model, file)

In [36]:
guardar_modelo(rf_model)

## PREDICCIONES

In [ ]:
columnas = [
    'pantalla_in', 
    
]

In [ ]:
datos_movil = {
    "ano":2025,
    "memoria": 100,
    "marca": 70,
    "pantalla_in": 6.7, 
    "pantalla_tipo": 2, 
    "velocidad_cpu_ghz": 6.2,
    "ram": 32, 
    "grosor": 0.77, 
    "peso":300,
    'ancho_px':1080, 
    'alto_px':2400,     
    "bateria": 5000,  
    'promedio_valoraciones':13566.95, 
    'precio_anterior':500,

}

# Año no responde bien
# Lo mes important para el preu es el preu anterior
# Bateria responde bien
# Marca responde bien
# RAM Responde bien
# CPU Responde bien
# Falta comprobar les dimensions


df_movil = pd.DataFrame([datos_movil])
# Convertir a DataFrame
df_movil = pd.DataFrame([datos_movil])

# Realizar la predicción
prediccion = rf_model.predict(df_movil)

print(f"Predicción del modelo: {prediccion[0]}")


Predicción del modelo: 382.3758000000001
